In [661]:
import numpy as np
import pandas as pd
import random 
import matplotlib.pyplot as plt
from scipy.stats import norm
from sys import maxsize
import math

In [704]:
#a=np.loadtxt(open("P1b_train_data_2D.csv", "rb"), delimiter=",")
a=np.loadtxt(open("P1b_train_data_2D.csv", "rb"), delimiter=",")
X=a[:,0:2]
X.shape

(200, 2)

In [663]:
a=np.loadtxt(open("P1b_train_data_2D.csv", "rb"), delimiter=",")
xs=a[:,0]
ys=a[:,1]
labels=a[:,2]
data = {'x': xs, 'y': ys, 'label': labels}
df = pd.DataFrame(data=data)

In [664]:
#Random Sampling


In [665]:
# ### Setup
# # set random seed
# rand.seed(42)

# # 2 clusters
# # not that both covariance matrices are diagonal
# mu1 = [0, 5]
# sig1 = [ [2, 0], [0, 3] ]

# mu2 = [5, 0]
# sig2 = [ [4, 0], [0, 1] ]

In [666]:
# # generate samples
# x1, y1 = np.random.multivariate_normal(mu1, sig1, 100).T
# x2, y2 = np.random.multivariate_normal(mu2, sig2, 100).T

# xs = np.concatenate((x1, x2))
# ys = np.concatenate((y1, y2))
# labels = ([1] * 100) + ([2] * 100)

# data = {'x': xs, 'y': ys, 'label': labels}
# df = pd.DataFrame(data=data)

In [667]:
# # inspect the data
# df.head()
# df.tail()

# fig = plt.figure()
# plt.scatter(data['x'], data['y'], 24, c=data['label'])
# fig.savefig("true-values.png")

In [668]:
### Expectation-maximization

# initial guesses - intentionally bad
guess = { 'mu1': [-1,-1],
          'sig1': [ [1.5, 0.5], [0.5, 1.5] ],
          'mu2': [5,3],
          'sig2': [ [2, 0], [0, 2] ],
          'lambda': [0.4, 0.6]
        }

In [669]:
pq=pd.DataFrame(guess)
np.array(pq['sig1'])

array([list([1.5, 0.5]), list([0.5, 1.5])], dtype=object)

In [670]:
# type(np.asarray([[parameters['sig1'][0][0],parameters['sig1'][0][1]], [parameters['sig1'][1][0],parameters['sig1'][1][1]]]))

In [671]:
# probability that a point came from a Guassian with given parameters
# note that the covariance must be diagonal for this to work
def prob(X_test, mean, var, lam):
    p =lam* pow(pow(2*math.pi,2)*np.linalg.det(var),-0.5)*pow(math.e,-0.5*np.matmul(np.matmul(X_test - mean,np.linalg.inv(var)),np.transpose(X_test - mean)))
    #print(p)
    return p

In [672]:
# assign every data point to its most likely cluster
# Classification
def expectation(dataFrame, parameters):
  for i in range(dataFrame.shape[0]):
    x = dataFrame['x'][i]
    y = dataFrame['y'][i]
    np.array([x,y])
    np.array([parameters['mu1'][0],parameters['mu1'][1]])
    np.array(parameters['lambda'][0])
    np.array([[parameters['sig1'][0][0],parameters['sig1'][0][1]], [parameters['sig1'][1][0],parameters['sig1'][1][1]]])
    
    p_cluster1 = prob(np.array([x,y]), np.array([parameters['mu1'][0],parameters['mu1'][1]]) , np.array([[parameters['sig1'][0][0],parameters['sig1'][0][1]], [parameters['sig1'][1][0],parameters['sig1'][1][1]]]), np.array(parameters['lambda'][0]) )
    p_cluster2 = prob(np.array([x,y]), [parameters['mu2'][0],parameters['mu2'][1]] ,np.array([[parameters['sig2'][0][0],parameters['sig2'][0][1]], [parameters['sig2'][1][0],parameters['sig2'][1][1]]]), parameters['lambda'][1] )
    if p_cluster1 > p_cluster2:
      dataFrame['label'][i] = 1
    else:
      dataFrame['label'][i] = 2
  return dataFrame

In [673]:
# update estimates of lambda, mu and sigma

def maximization(dataFrame, parameters):
  points_assigned_to_cluster1 = dataFrame[dataFrame['label'] == 1]
  #print('points_assigned_to_cluster1', points_assigned_to_cluster1)
  points_assigned_to_cluster2 = dataFrame[dataFrame['label'] == 2]
  #print('points_assigned_to_cluster2', points_assigned_to_cluster2)
  percent_assigned_to_cluster1 = len(points_assigned_to_cluster1) / float(len(dataFrame))
  percent_assigned_to_cluster2 = 1 - percent_assigned_to_cluster1
  parameters['lambda'] = [percent_assigned_to_cluster1, percent_assigned_to_cluster2 ]
  parameters['mu1'] = [points_assigned_to_cluster1['x'].mean(), points_assigned_to_cluster1['y'].mean()]
  #print("parameters['mu1']",parameters['mu1'])  
  parameters['mu2'] = [points_assigned_to_cluster2['x'].mean(), points_assigned_to_cluster2['y'].mean()]
  parameters['sig1'] = np.cov(points_assigned_to_cluster1['x'],points_assigned_to_cluster1['y']).tolist()
  #print("points_assigned_to_cluster1['x']\n")
  #print("parameters['sig1']\n",parameters['sig1']) 
  parameters['sig2'] = np.cov(points_assigned_to_cluster2['x'],points_assigned_to_cluster2['y']).tolist() 
  return parameters,points_assigned_to_cluster1['x'],points_assigned_to_cluster1['y']

In [674]:
# get the distance between points
# used for determining if params have converged
def distance(old_params, new_params):
  dist = 0
  for param in ['mu1', 'mu2']:
    for i in range(len(old_params)):
      dist += (old_params[param][i] - new_params[param][i]) ** 2
  return dist ** 0.5

In [675]:
# loop until parameters converge
shift = maxsize
epsilon = 0.01
iters = 0
df_copy = df.copy()
# randomly assign points to their initial clusters
df_copy['label'] = map(lambda x: x+1, np.random.choice(2, len(df)))
params = pd.DataFrame(guess)

while shift > epsilon:
  iters += 1
  # E-step
  updated_labels = expectation(df_copy.copy(), params)

  # M-step
  updated_parameters,xx,yy = maximization(updated_labels, params.copy())

  # see if our estimates of mu have changed
  # could incorporate all params, or overall log-likelihood
  shift = distance(params, updated_parameters)

  # logging
  print("iteration {}, shift {}".format(iters, shift))
  #print(345)

  # update labels and params for the next iteration
  df_copy = updated_labels
  #print(4)
  params = updated_parameters
  print(params)
  #print(5)
  params
  #print(4)

#   fig = plt.figure()
#   plt.scatter(df_copy['x'], df_copy['y'], 24, c=df_copy['label'])
#   fig.savefig("iteration{}.png".format(iters))

C:\Users\range\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\range\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


iteration 1, shift 2.426806809751528
        mu1                                      sig1       mu2  \
0 -0.045485   [1.025231650381864, 0.3620614113928044]  2.996989   
1 -0.092037  [0.3620614113928044, 0.9214867173995376]  3.376620   

                                         sig2  lambda  
0  [0.9537728997835376, -0.14952339315278593]    0.49  
1  [-0.14952339315278593, 2.5091563507008336]    0.51  
iteration 2, shift 0.08647745466027754
        mu1                                       sig1       mu2  \
0 -0.086675  [0.9297345722105262, 0.43999921305263173]  2.976502   
1 -0.085308  [0.43999921305263173, 0.8565601765614033]  3.303704   

                                         sig2  lambda  
0  [0.9873688475690808, -0.12189106117251675]    0.48  
1   [-0.12189106117251675, 2.809928645421958]    0.52  
iteration 3, shift 0.0
        mu1                                       sig1       mu2  \
0 -0.086675  [0.9297345722105262, 0.43999921305263173]  2.976502   
1 -0.085308  [0.439999

# Testing

In [676]:
params

,mu1,sig1,mu2,sig2,lambda
0,-0.086675,"[0.9297345722105262, 0.43999921305263173]",2.976502,"[0.9873688475690808, -0.12189106117251675]",0.48
1,-0.085308,"[0.43999921305263173, 0.8565601765614033]",3.303704,"[-0.12189106117251675, 2.809928645421958]",0.52


In [677]:
np.cov(xx,yy).tolist()

[[0.9297345722105262, 0.43999921305263173],
 [0.43999921305263173, 0.8565601765614033]]

In [678]:
abc=pd.DataFrame(guess)
np.matrix(abc['sig1'])
type([abc['sig2'][0],abc['sig2'][1]])
[abc['sig2'][0],abc['sig2'][1]]

[[2, 0], [0, 2]]

In [679]:
guess['sig2']

[[2, 0], [0, 2]]

In [680]:
params['sig1']

0    [0.9297345722105262, 0.43999921305263173]
1    [0.43999921305263173, 0.8565601765614033]
Name: sig1, dtype: object

In [681]:
np.array([params['sig1'][0],params['sig1'][1]])

array([[0.92973457, 0.43999921],
       [0.43999921, 0.85656018]])

In [682]:
type([[2,3],[4,5]])

list

In [683]:
lam=0.5
var=np.array([[2,3],[4,5]])
mean=np.array([6,5])
X_test=np.array([5,6])


In [684]:
X_test-mean
np.linalg.inv(var)
np.matmul(X_test - mean,np.linalg.inv(var))
np.transpose(X_test - mean)
pow(0.5,np.matmul(np.matmul(X_test - mean,np.linalg.inv(var)),np.transpose(X_test - mean)))

128.0

In [685]:
p =lam* pow(pow(2*math.pi,2)*np.linalg.det(var),-0.5)*pow(math.e,-0.5*np.matmul(np.matmul(X_test - mean,np.linalg.inv(var)),np.transpose(X_test - mean)))
    

C:\Users\range\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.


In [686]:
np.array(params['sig1'])

array([list([0.9297345722105262, 0.43999921305263173]),
       list([0.43999921305263173, 0.8565601765614033])], dtype=object)

In [687]:
np.array([[params['sig1'][0][0],params['sig1'][0][1]], [params['sig1'][1][0],params['sig1'][1][1]]])
   

array([[0.92973457, 0.43999921],
       [0.43999921, 0.85656018]])

# EM Bayes Classifier

In [688]:
params

,mu1,sig1,mu2,sig2,lambda
0,-0.086675,"[0.9297345722105262, 0.43999921305263173]",2.976502,"[0.9873688475690808, -0.12189106117251675]",0.48
1,-0.085308,"[0.43999921305263173, 0.8565601765614033]",3.303704,"[-0.12189106117251675, 2.809928645421958]",0.52


In [724]:
b_scre=0
atest=np.loadtxt(open("P1b_test_data_2D.csv", "rb"), delimiter=",")

for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, np.array([params['mu1'][0],params['mu1'][1]]) , np.array([[params['sig1'][0][0],params['sig1'][0][1]], [params['sig1'][1][0],params['sig1'][1][1]]]),1)
    f_1 = prob(X_test, np.array([params['mu2'][0],params['mu2'][1]]) , np.array([[params['sig2'][0][0],params['sig2'][0][1]], [params['sig2'][1][0],params['sig2'][1][1]]]), 1) 
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

97.5

# Bayes Classifier

In [718]:
b_scre=0
a=np.loadtxt(open("P1b_train_data_2D.csv", "rb"), delimiter=",")
atest=np.loadtxt(open("P1b_test_data_2D.csv", "rb"), delimiter=",")
X_mean1 = np.mean(a[0:100,0:2],axis=0)
X_mean2 = np.mean(a[100:200,0:2],axis=0)
X_var1 = np.cov(a[0:100,0],a[0:100,1]).tolist()
X_var2 = np.cov(a[100:200,0],a[100:200,1]).tolist()
for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, X_mean1, X_var1,1)
    f_1 = prob(X_test, X_mean2, X_var2,1)
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

97.5

In [723]:
X_var2


[[0.9811784625898988, -0.16080762704646465],
 [-0.16080762704646465, 2.8730789192767667]]

In [692]:
c=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")
Xc1=a[0:100,0:2]
Xc2=a[100:200,0:2]


In [725]:
b_scre=0
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")
X_mean1 = np.mean(c[0:100,0:2],axis=0)
X_mean2 = np.mean(c[100:200,0:2],axis=0)
X_var1 = np.cov(c[0:100,0],c[0:100,1]).tolist()
lamc1=1/X_mean2[0]
lamc2=1/X_mean2[1]
for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, X_mean1, X_var1,1)
    f_1 = lamc1*lamc2*pow(math.e,-lamc1*X_test[0]-lamc2*X_test[1])
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

84.0

# Varying Sample

In [736]:
c1=list(range(0,100))
c2=list(range(100,200))
X1_100=X[random.sample(c1,100)]
X2_100=X[random.sample(c2,100)]
X1_100_mean=np.mean(X1_100,axis=0)
X2_100_mean=np.mean(X2_100,axis=0)
X1_100_var=np.matmul(np.transpose(X1_100-np.ones((100,1))*X1_100_mean),(X1_100-np.ones((100,1))*X1_100_mean))/100
X2_100_var=np.matmul(np.transpose(X2_100-np.ones((100,1))*X1_100_mean),(X2_100-np.ones((100,1))*X2_100_mean))/100
X_100=np.concatenate((X1_100,X2_100))
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")

In [737]:
b_scre=0
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")
#X_mean1 = np.mean(a[0:100,0:2],axis=0)
#X_mean2 = np.mean(a[100:200,0:2],axis=0)
#X_var1 = np.cov(a[0:100,0],a[0:100,1]).tolist()
#X_var2 = np.cov(a[100:200,0],a[100:200,1]).tolist()
for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, X1_100_mean, X1_100_var,1)
    f_1 = prob(X_test, X2_100_mean, X2_100_var,1)
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

98.0

In [705]:
c1=list(range(0,100))
c2=list(range(100,200))
X1_5=X[random.sample(c1,5)]
X2_5=X[random.sample(c2,5)]
X1_5_mean=np.mean(X1_5,axis=0)
X2_5_mean=np.mean(X2_5,axis=0)
X1_5_var=np.matmul(np.transpose(X1_5-np.ones((5,1))*X1_5_mean),(X1_5-np.ones((5,1))*X1_5_mean))/5
X2_5_var=np.matmul(np.transpose(X2_5-np.ones((5,1))*X1_5_mean),(X2_5-np.ones((5,1))*X2_5_mean))/5
X_5=np.concatenate((X1_5,X2_5))
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")


In [706]:
b_scre=0
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")
#X_mean1 = np.mean(a[0:100,0:2],axis=0)
#X_mean2 = np.mean(a[100:200,0:2],axis=0)
#X_var1 = np.cov(a[0:100,0],a[0:100,1]).tolist()
#X_var2 = np.cov(a[100:200,0],a[100:200,1]).tolist()
for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, X1_5_mean, X1_5_var,1)
    f_1 = prob(X_test, X2_5_mean, X2_5_var,1)
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

93.0

In [707]:
c1=list(range(0,100))
c2=list(range(100,200))
X1_10=X[random.sample(c1,10)]
X2_10=X[random.sample(c2,10)]
X1_10_mean=np.mean(X1_10,axis=0)
X2_10_mean=np.mean(X2_10,axis=0)
X1_10_var=np.matmul(np.transpose(X1_10-np.ones((10,1))*X1_10_mean),(X1_10-np.ones((10,1))*X1_10_mean))/10
X2_10_var=np.matmul(np.transpose(X2_10-np.ones((10,1))*X1_10_mean),(X2_10-np.ones((10,1))*X2_10_mean))/10
X_10=np.concatenate((X1_10,X2_10))
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")

In [708]:
b_scre=0
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")
#X_mean1 = np.mean(a[0:100,0:2],axis=0)
#X_mean2 = np.mean(a[100:200,0:2],axis=0)
#X_var1 = np.cov(a[0:100,0],a[0:100,1]).tolist()
#X_var2 = np.cov(a[100:200,0],a[100:200,1]).tolist()
for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, X1_10_mean, X1_10_var,1)
    f_1 = prob(X_test, X2_10_mean, X2_10_var,1)
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

98.0

In [709]:
c1=list(range(0,100))
c2=list(range(100,200))
X1_25=X[random.sample(c1,25)]
X2_25=X[random.sample(c2,25)]
X1_25_mean=np.mean(X1_25,axis=0)
X2_25_mean=np.mean(X2_25,axis=0)
X1_25_var=np.matmul(np.transpose(X1_25-np.ones((25,1))*X1_25_mean),(X1_25-np.ones((25,1))*X1_25_mean))/25
X2_25_var=np.matmul(np.transpose(X2_25-np.ones((25,1))*X1_25_mean),(X2_25-np.ones((25,1))*X2_25_mean))/25
X_25=np.concatenate((X1_25,X2_25))
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")

In [710]:
b_scre=0
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")
#X_mean1 = np.mean(a[0:100,0:2],axis=0)
#X_mean2 = np.mean(a[100:200,0:2],axis=0)
#X_var1 = np.cov(a[0:100,0],a[0:100,1]).tolist()
#X_var2 = np.cov(a[100:200,0],a[100:200,1]).tolist()
for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, X1_25_mean, X1_25_var,1)
    f_1 = prob(X_test, X2_25_mean, X2_25_var,1)
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

98.5

In [711]:
c1=list(range(0,100))
c2=list(range(100,200))
X1_75=X[random.sample(c1,75)]
X2_75=X[random.sample(c2,75)]
X1_75_mean=np.mean(X1_75,axis=0)
X2_75_mean=np.mean(X2_75,axis=0)
X1_75_var=np.matmul(np.transpose(X1_75-np.ones((75,1))*X1_75_mean),(X1_75-np.ones((75,1))*X1_75_mean))/75
X2_75_var=np.matmul(np.transpose(X2_75-np.ones((75,1))*X1_75_mean),(X2_75-np.ones((75,1))*X2_75_mean))/75
X_75=np.concatenate((X1_75,X2_75))
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")

In [712]:
b_scre=0
atest=np.loadtxt(open("P1c_test_data_2D.csv", "rb"), delimiter=",")
#X_mean1 = np.mean(a[0:100,0:2],axis=0)
#X_mean2 = np.mean(a[100:200,0:2],axis=0)
#X_var1 = np.cov(a[0:100,0],a[0:100,1]).tolist()
#X_var2 = np.cov(a[100:200,0],a[100:200,1]).tolist()
for j in range(200):
    
    X_test=atest[j,0:2] ##
    Y_test=atest[j,2] ##
    f1 = prob(X_test, X1_75_mean, X1_75_var,1)
    f_1 = prob(X_test, X2_75_mean, X2_75_var,1)
    if f1>f_1:
        b_clss=1
    else:
        b_clss=-1
    if Y_test==b_clss:
        b_scre=b_scre+1
b_scre/len(X_test)

98.5

In [702]:
X


array([[ 1.3630e-01,  5.8610e-01],
       [ 1.5985e+00,  7.5680e-01],
       [-1.7900e-02,  4.1790e-01],
       [-2.7473e+00, -9.5750e-01],
       [ 3.3270e-01,  1.1452e+00],
       [ 3.3460e-01,  1.6193e+00],
       [-4.9160e-01, -9.4300e-01],
       [ 1.5934e+00,  1.9558e+00],
       [-7.2300e-01, -5.9910e-01],
       [ 5.8580e-01,  1.1473e+00],
       [ 6.0070e-01, -1.1334e+00],
       [ 1.6615e+00,  4.0630e-01],
       [-1.4250e+00, -2.5512e+00],
       [ 1.8110e-01,  7.7310e-01],
       [ 1.2950e-01, -5.5600e-01],
       [-1.2880e-01, -1.2945e+00],
       [ 2.4630e-01, -6.4220e-01],
       [-1.0845e+00, -5.7980e-01],
       [ 9.6030e-01, -4.2780e-01],
       [-9.3440e-01, -9.3800e-02],
       [ 1.6610e-01,  1.5000e+00],
       [-1.0224e+00, -4.6170e-01],
       [-2.7990e-01,  7.2000e-03],
       [ 1.6790e-01,  9.0970e-01],
       [ 2.8180e-01, -1.0590e-01],
       [ 1.2080e-01,  2.2030e-01],
       [ 1.1254e+00,  1.6780e-01],
       [ 1.2100e-01,  5.2610e-01],
       [ 1.8294e+00,

In [703]:
xs

array([ 0.1363,  1.5985, -0.0179, -2.7473,  0.3327,  0.3346, -0.4916,
        1.5934, -0.723 ,  0.5858,  0.6007,  1.6615, -1.425 ,  0.1811,
        0.1295, -0.1288,  0.2463, -1.0845,  0.9603, -0.9344,  0.1661,
       -1.0224, -0.2799,  0.1679,  0.2818,  0.1208,  1.1254,  0.121 ,
        1.8294, -0.2177, -0.7404,  1.5184, -1.8561,  0.9285, -0.2455,
       -0.4754,  0.6396, -1.9128, -0.5026, -0.8565,  1.9769, -1.5365,
       -0.0102, -1.4673,  0.6168,  1.4464,  1.1459,  0.3315, -1.6697,
       -1.2124,  1.5791, -0.889 , -1.023 ,  0.0509,  1.218 ,  0.0195,
       -1.3425, -0.9078, -0.509 ,  0.9052, -1.1312, -0.2473,  2.4899,
       -0.2414, -1.1403, -1.6333,  0.4166, -0.2761,  0.6549,  0.5282,
        0.5087, -0.5726,  1.5016,  1.6246, -0.3409,  0.4568,  0.0542,
       -0.3463,  1.6839, -0.6443, -0.1616,  0.9958,  0.2292, -0.5671,
        0.6358, -1.402 , -0.0493, -1.2985,  2.3744,  0.1835, -0.4063,
        1.4856, -1.2181, -0.3053, -0.7423, -0.9748, -0.3296,  0.0945,
        0.4431, -0.8